## Import icbhi-preprocessing-v2 and icbhi-extraction-v2 data before running

# Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.cm as cm
import pandas as pd

# Not immediately usable with this dataset as files contain 24-bit data
from scipy.io import wavfile
from scipy.io.wavfile import read, write

import IPython.display as ipd
from IPython.display import Audio
from numpy.fft import fft, ifft

import librosa as lr
import librosa.display

import os
import pickle
from glob import glob

import seaborn as sns

from scipy import signal as sig
from scipy.signal import butter, lfilter, sosfilt
from tqdm import tqdm
import pywt
import random
import skimage.data
from skimage.restoration import denoise_wavelet
from pywt import swt
import time as tm
import collections

# if ('pydub' in globals()) == False:
#   !pip install pydub
# from pydub import AudioSegment
# from pydub.utils import make_chunks


# LT-06-24: import io to save intermediate outputs; use "as" to shorten module.function name as "spio"
import scipy.io as spio


# LT-06-24: install modules if not found 
# if ( 'umap' in globals() ) == False: 
#   !pip install umap-learn

import umap.umap_ as umap

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, minmax_scale
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_samples, silhouette_score, classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, roc_curve

print('Importing done')

# Feature & Class Data

In [ ]:
class_list = np.load("../input/icbhi-preprocessing-v3/class_list.npy")
directory = '../input/icbhi-extraction-v3/' 
nsamples = len(class_list)
features = np.zeros( ( nsamples, 97573) )
classes = np.zeros( nsamples ).astype(int)

for d in range( 0, nsamples):
    try: # quick hack to read some        
        classes[d] = class_list[d] 
        features[d] = np.load(directory + str(d) + '_features.npy')
    except:
        break

# Subsetting

In [ ]:
normals = np.where(classes == 0)[0].tolist()
crackles = np.where(classes == 1)[0].tolist()
wheezes = np.where(classes == 2)[0].tolist()
cracklewheezes = np.where(classes == 3)[0].tolist()

i_subsets = []
n = 500 # samples per class
seeds = [3319, 1339, 136]
for i, s in enumerate(seeds):
    random.seed(s)
    set1 = random.sample(normals, n)
    set2 = random.sample(crackles, n)
    set3 = random.sample(wheezes, n)
    set4 = random.sample(cracklewheezes, n)
    i_subsets.append(np.hstack((set1, set2, set3, set4)))
    
def get_feature_set(l, features):
    f = np.zeros((len(l), 97573))
    for i, index in enumerate(l):
        f[i] = features[index]
    return f

feature_sets = []
for l in i_subsets:
    feature_sets.append(get_feature_set(l, features))

class_set = [0] * n
class_set.extend([1] * n)
class_set.extend([2] * n)
class_set.extend([3] * n)
    
feature_sets

In [ ]:
print(len(normals))
print(len(crackles))
print(len(wheezes))
print(len(cracklewheezes))

# K-Nearest Neighbours

In [ ]:
m = min(len(normals), len(crackles), len(wheezes), len(cracklewheezes))

normals = normals[:m]
crackles = crackles[:m]
wheezes = wheezes[:m]
cracklewheezes = cracklewheezes[:m]
balanced_set = np.hstack((normals, crackles, wheezes, cracklewheezes))

balanced_set

In [ ]:
balanced_features = np.zeros((len(balanced_set), 97573))
for i, index in enumerate(balanced_set):
    balanced_features[i] = features[index]

In [ ]:
balanced_classes = [0] * m
balanced_classes.extend([1] * m)
balanced_classes.extend([2] * m)
balanced_classes.extend([3] * m)

In [ ]:
# 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(balanced_features, balanced_classes, random_state=321)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)
y_pred

In [ ]:
plot_confusion_matrix(knn, X_test, y_test)

In [ ]:
accuracy_score(y_test, y_pred)

# Dry Run

In [ ]:
expert = '../input/dryrun/9002_expert/'
expertnames = [s.split('.')[0] for s in os.listdir(path = expert) if '.wav' in s]
nonexpert = '../input/dryrun/9002_nonexpert/'
nonexpertnames = [s.split('.')[0] for s in os.listdir(path = nonexpert) if '.wav' in s]
volunteer = '../input/dryrun/9002_volunteer/'
volunteernames = [s.split('.')[0] for s in os.listdir(path = volunteer) if '.wav' in s]

In [ ]:
# Get waveforms
def signal_and_sr(filename, root):
    name = root + filename + '.wav'
    signal, sr = lr.load(name, sr=None)
    return signal, sr

def waveforms(files, root):
  signals = []
  srs = []
  for name in files:
    (sig, sr) = signal_and_sr(name, root) # signal is an array with (sr * duration) values
    # if (len(sig) % 2) == 0: # signal array must be an even number of values for MODWT
    signals.append(sig)
    srs.append(sr)
  return signals, srs

In [ ]:
print( expert )
expertnames


In [ ]:
exp_sigs, exp_srs = waveforms(expertnames, expert)
nonexp_sigs, nonexp_srs = waveforms(nonexpertnames, nonexpert)
vol_sigs, vol_srs = waveforms(volunteernames, volunteer)
for i, s in enumerate(exp_sigs):
    print(lr.get_duration(y = s, sr = exp_srs[i]))
for i, s in enumerate(nonexp_sigs):
    print(lr.get_duration(y = s, sr = nonexp_srs[i]))
for i, s in enumerate(vol_sigs):
    print(lr.get_duration(y = s, sr = vol_srs[i]))

In [ ]:
def preprocessing(samples, files, sr=4000, max_length=6, new_sr=8000, lowcut=150, highcut=2000, order=5):
    """
    1) Add zeros to slices so that they are the same length or reduce their length to the maximum length
    2) Apply bandpass filters and resample to 8000Hz
    3) Scale signal amplitudes to be between -1 and 1
    """
    # Zero-padding
    padded_slices = []
    for i, signal in enumerate(samples):
        if (lr.get_duration(signal, sr=sr) <= 15.0) & (lr.get_duration(signal, sr=sr) > 10.0):
            slices = [signal[:int(round(5 * sr))], signal[int(round(5 * sr)):int(round(10 * sr))], signal[int(round(10 * sr)):]]
            for s in slices:
                padded_slices.append(lr.util.pad_center(s, max_length*sr))
        elif (lr.get_duration(signal, sr=sr) <= 10.0) & (lr.get_duration(signal, sr=sr) > 5.0):
            slices = [signal[:int(round(5 * sr))], signal[int(round(5 * sr)):]]
            for s in slices:
                padded_slices.append(lr.util.pad_center(s, max_length*sr))
        else:
            padded_slices.append(lr.util.pad_center(signal, max_length*sr))
    
    # Bandpass filter & resample (technically should be resampling after filtering but this causes a bug...)
    processed = np.zeros((len(padded_slices), (max_length*new_sr)))
    for i, signal in enumerate(padded_slices):
        resample = lr.resample(signal, sr, new_sr)
        y = butter_bandpass_filter(resample, lowcut, highcut, new_sr, order) # dataset contains more than one sr - 44100, 4000 and 10000; Pramono et al. recommend 8000Hz as respiratory sound information is bandlimited up to 2000Hz..?
        processed[i, :len(y)] = y
    
    # Amplitude scaling to [-1, 1]
    for i, signal in enumerate(processed):
        peak = max(abs(signal))
        dB = 0

        amp_lin = 10**(dB/20)
        z = amp_lin*(1/peak)*signal
        
        processed[i, :len(z)] = z
        
    return processed

def butter_bandpass_filter(data, lowcut, highcut, sr, order):
    sos = butter_bandpass(lowcut, highcut, sr, order=order)
    y = sosfilt(sos, data)
    return y
    
def butter_bandpass(lowcut, highcut, sr, order):
    nyq = 0.5 * sr
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

In [ ]:
prepro_exp = preprocessing(exp_sigs, expertnames)
prepro_nonexp = preprocessing(nonexp_sigs, nonexpertnames)
prepro_vol = preprocessing(vol_sigs, volunteernames)

In [ ]:
def extract(signal, sr=8000):
    """
    signal: slice that has already undergone preprocessing
    sr: sampling rate of slice
    """
    # Spectrograms
    stft = lr.stft(signal)
    spec = lr.amplitude_to_db(abs(stft))
    
    # Mel-cepstral frequency coefficients
    mfcc = lr.feature.mfcc(signal, sr, n_mfcc=13)
    
    # Wavelet transform using Baubechies with 4 vanishing moments
    modwt = np.array(pywt.dwt(signal, 'db4', mode='zero'))
    # Zero crossing rate
    wt_zcr = len(np.nonzero(np.diff(np.array(modwt) > 0))[0])
    
    return spec, mfcc, wt_zcr

def feature_extraction(preprocessed_slices):
    feature_set = np.zeros((len(preprocessed_slices), 97573))
    for i, signal in enumerate(preprocessed_slices):
        # Extract
        spec, mfcc, wt_zcr = extract(signal)
        
        # Reshape
        a,b = spec.shape
        c,d = mfcc.shape
        spec = spec.reshape((a*b))
        mfcc = mfcc.reshape((c*d))
        features = np.hstack((spec, mfcc, wt_zcr))
        feature_set[i] = features
        
    return feature_set

In [ ]:
print( len(exp_sigs), prepro_exp.shape ) # 13 x 3 ?
print( len(nonexp_sigs), prepro_nonexp.shape ) # 13 x 3 = 39 
print( len(vol_sigs), prepro_vol.shape ) # 6 x 3 = 18 

In [ ]:
exp_features = feature_extraction(prepro_exp)
nonexp_features = feature_extraction(prepro_nonexp)
vol_features = feature_extraction(prepro_vol)

In [ ]:
exp_pred = knn.predict(exp_features)
nonexp_pred = knn.predict(nonexp_features)
vol_pred = knn.predict(vol_features)

In [ ]:
print(exp_pred)
print(nonexp_pred)
print(vol_pred)

In [ ]:
with open('/input/knn_4.pkl', 'wb') as f:
    pickle.dump(knn, f)

In [ ]:
expertnames

In [ ]:

def write_to_pkl( file, dic ):    
    from pickle import dump        
    f=open( file + '.pkl' , 'wb')      
    dump( dic, f)
    f.close()
    
    
    
def read_from_pkl( file ):    
    from pickle import load    
    file = file.replace( '.pkl','')    
    file = open( file + '.pkl', 'rb')    
    try:
        dat = load( file )
    except:
        dat = load( file, encoding='latin1')
    return dat

knn4=read_from_pkl('knn_4.pkl')


In [ ]:
# knn4's predictions are reproducible 

exp_pred_knn4 = knn4.predict(exp_features)
exp_pred_knn4

In [ ]:

# prepro_exp = preprocessing(exp_sigs, expertnames)

expertnames
exp_sigs[1].shape

In [ ]:

plt.plot( exp_features[1,:-1] )

# Model Training

In [ ]:
m = 4 # Accuracies per class
KNNs = {}
acc = np.zeros( (len(feature_sets), m) )
knn = KNeighborsClassifier(n_neighbors=8) # Change n_neighbors to desired k

for i, s in enumerate(feature_sets):
    X_train, X_test, y_train, y_test = train_test_split(s, class_set, random_state=321)
    KNNs[i] = knn.fit(X_train, y_train)
    y_pred = KNNs[i].predict(X_test)
    matrix = confusion_matrix(y_test, y_pred)
    acc[i] = matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
acc

# Decision Tree

In [ ]:
m = 4 # Accuracies per class
DTs = {}
acc = np.zeros( (len(feature_sets), m) )
dt = DecisionTreeClassifier(criterion="entropy", max_depth=9)

for i, s in enumerate(feature_sets):
    X_train, X_test, y_train, y_test = train_test_split(s, class_set, random_state=321)
    DTs[i] = dt.fit(X_train, y_train)
    y_pred = DTs[i].predict(X_test)
    matrix = confusion_matrix(y_test, y_pred)
    acc[i] = matrix.diagonal()/matrix.sum(axis=1)

In [ ]:
acc

In [ ]:
max_depths = np.linspace(1, 20, 20, endpoint=True)
X_train, X_test, y_train, y_test = train_test_split(feature_sets[0], class_set, random_state=321)

train = []
test = []
for depth in max_depths:
    dt = DecisionTreeClassifier(max_depth=depth)
    dt.fit(X_train, y_train)
    
    train_pred = dt.predict(X_train)
    
    matrix = confusion_matrix(y_train, train_pred)
    acc = np.mean(matrix.diagonal()/matrix.sum(axis=1))
    train.append(acc)
    
    y_pred = dt.predict(X_test)
    
    matrix = confusion_matrix(y_test, y_pred)
    acc = np.mean(matrix.diagonal()/matrix.sum(axis=1))
    test.append(acc)
    
line1, = plt.plot(max_depths, train, label="Train Accuracy")
line2, = plt.plot(max_depths, test, label="Test Accuracy")
    
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel("Average Classwise Accuracy")
plt.xlabel("Tree Depth")
plt.show()